In [1]:
import pandas as pd
import numpy as np

def encrypt(pt,key):

  df = pd.DataFrame(columns=[0,1,2,3,4])

  for col in df.columns:
    df[col]=[" " for i in range(5)]

  row=0
  col=0
  p=""
  for char in key:
    if char not in p:
      p=p+char

  for i in p:
    df[col][row]=i
    if(col<4):
      col=col+1
    else:
      row=row+1
      col=0

  alphabets=[]
  for i in range(ord('a'),ord('z')+1):
    alphabets.append(chr(i))

  if('i' in p or 'j' in p):
    alphabets.remove('i')
    alphabets.remove('j')

  newL=[]
  for alpha in alphabets:
      if(alpha not in p):
          newL.append(alpha)

  if('i' in newL):
    newL.remove('j')

  for letter in newL:
    df[col][row]=letter
    if(col<4):
      col=col+1
    else:
      row=row+1
      col=0

  pairs=[]

  i=0
  while i<len(pt):
    try:
      if(pt[i]==pt[i+1]):
        pair = pt[i] + "x"
        i=i+1
        pairs.append(pair)
      else:
        pair=pt[i]+pt[i+1]
        i=i+2
        pairs.append(pair)
    except(IndexError):
      pair = pt[i] + "x"
      pairs.append(pair)
      break

  ct=""
  for i in pairs:
    res1=np.where(df==i[0])
    res2=np.where(df==i[1])

    row1=res1[0][0]
    col1=res1[1][0]

    row2=res2[0][0]
    col2=res2[1][0]

    if(col1==col2):
      if(row1==4):
        row1=0
      else:
        row1=row1+1
      if(row2==4):
        row2=0
      else:
        row2=row2+1

    elif(row1==row2):
      if(col1==4):
        col1=0
      else:
        col1=col1+1
      if(col2==4):
        col2=0
      else:
        col2=col2+1

    else:
      temp=col1
      col1=col2
      col2=temp

    l1=df[col1][row1]
    l2=df[col2][row2]
    newPair=l1+l2
    ct=ct+newPair

  return ct, df

ct,df=encrypt('instruments','monarchy')
ct

'gatlmzclrqxa'

In [2]:
df

,0,1,2,3,4
0,m,o,n,a,r
1,c,h,y,b,d
2,e,f,g,i,k
3,l,p,q,s,t
4,u,v,w,x,z


In [2]:
def create_playfair_key_matrix(key):
    # Create a matrix and a set to keep track of added letters
    matrix = [[None for _ in range(5)] for _ in range(5)]
    added = set()

    # Prepare the key: remove duplicates and spaces, and replace J with I
    prepared_key = ''
    for char in key.upper():
        if char not in added and char != ' ':
            prepared_key += 'I' if char == 'J' else char
            added.add(char)

    # Fill the matrix with the key characters
    alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
    key_index, alpha_index = 0, 0
    for i in range(5):
        for j in range(5):
            if key_index < len(prepared_key):
                matrix[i][j] = prepared_key[key_index]
                key_index += 1
            else:
                # Add remaining alphabet characters
                while alphabet[alpha_index] in added:
                    alpha_index += 1
                matrix[i][j] = alphabet[alpha_index]
                added.add(alphabet[alpha_index])
                alpha_index += 1

    return matrix

def playfair_cipher_encrypt(plaintext, key):
    # Create the key matrix
    key_matrix = create_playfair_key_matrix(key)

    # Preprocess plaintext: Replace J with I, split into digraphs
    prepared_text = ""
    for char in plaintext.upper():
        if char != ' ':
            prepared_text += 'I' if char == 'J' else char

    # Split into digraphs, adding 'X' if needed
    digraphs = []
    i = 0
    while i < len(prepared_text):
        a = prepared_text[i]
        b = prepared_text[i + 1] if i + 1 < len(prepared_text) else 'X'
        if a == b:
            b = 'X'
            i -= 1  # To handle the added X in the next iteration
        digraphs.append(a + b)
        i += 2

    # Function to find the position of a letter in the key matrix
    def find_position(letter):
        for i, row in enumerate(key_matrix):
            for j, char in enumerate(row):
                if char == letter:
                    return i, j

    # Encrypt each digraph
    encrypted_text = ''
    for digraph in digraphs:
        row1, col1 = find_position(digraph[0])
        row2, col2 = find_position(digraph[1])

        if row1 == row2:  # Same row
            col1 = (col1 + 1) % 5
            col2 = (col2 + 1) % 5
        elif col1 == col2:  # Same column
            row1 = (row1 + 1) % 5
            row2 = (row2 + 1) % 5
        else:  # Rectangle
            col1, col2 = col2, col1

        encrypted_text += key_matrix[row1][col1] + key_matrix[row2][col2]

    return encrypted_text, key_matrix

# Example usage
plaintext = "HELLO WORLD"
key = "MONARCHY"
encrypted_text, key_matrix = playfair_cipher_encrypt(plaintext, key)

# Print the key matrix and the encrypted text
encrypted_text, key_matrix

('CFSUPMVNMTBZ',
 [['M', 'O', 'N', 'A', 'R'],
  ['C', 'H', 'Y', 'B', 'D'],
  ['E', 'F', 'G', 'I', 'K'],
  ['L', 'P', 'Q', 'S', 'T'],
  ['U', 'V', 'W', 'X', 'Z']])

In [3]:
#Playfair Cipher
def prepare_text(text):
    text = text.upper().replace(" ", "")
    text = text.replace("J", "I")
    return text

def generate_key_table(key):
    key = prepare_text(key)
    key_table = []
    for char in key:
        if char not in key_table:
            key_table.append(char)
    alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
    for char in alphabet:
        if char not in key_table:
            key_table.append(char)
    return key_table

def construct_playfair_matrix(key):
    key_table = generate_key_table(key)
    matrix = [[0] * 5 for _ in range(5)]
    k = 0
    for i in range(5):
        for j in range(5):
            matrix[i][j] = key_table[k]
            k += 1
    return matrix

def find_char_positions(matrix, char):
    for i in range(5):
        for j in range(5):
            if matrix[i][j] == char:
                return i, j

def encrypt(text, key):
    matrix = construct_playfair_matrix(key)
    text = prepare_text(text)
    i = 0
    while i < len(text) - 1:
        if text[i] == text[i + 1]:
            text = text[:i + 1] + 'X' + text[i + 1:]
        i += 2
    if len(text) % 2 != 0:
        text += 'X'
    encrypted_text = ""
    for i in range(0, len(text), 2):
        char1, char2 = text[i], text[i + 1]
        row1, col1 = find_char_positions(matrix, char1)
        row2, col2 = find_char_positions(matrix, char2)
        if row1 == row2:
            encrypted_text += matrix[row1][(col1 + 1) % 5] + matrix[row2][(col2 + 1) % 5]
        elif col1 == col2:
            encrypted_text += matrix[(row1 + 1) % 5][col1] + matrix[(row2 + 1) % 5][col2]
        else:
            encrypted_text += matrix[row1][col2] + matrix[row2][col1]
    return encrypted_text

def decrypt(encrypted_text, key):
    matrix = construct_playfair_matrix(key)
    decrypted_text = ""
    for i in range(0, len(encrypted_text), 2):
        char1, char2 = encrypted_text[i], encrypted_text[i + 1]
        row1, col1 = find_char_positions(matrix, char1)
        row2, col2 = find_char_positions(matrix, char2)
        if row1 == row2:
            decrypted_text += matrix[row1][(col1 - 1) % 5] + matrix[row2][(col2 - 1) % 5]
        elif col1 == col2:
            decrypted_text += matrix[(row1 - 1) % 5][col1] + matrix[(row2 - 1) % 5][col2]
        else:
            decrypted_text += matrix[row1][col2] + matrix[row2][col1]
    return decrypted_text

key = "MONARCHY"
plaintext = "ATTACK"
encrypted_text = encrypt(plaintext, key)
print("Encrypted text:", encrypted_text)
decrypted_text = decrypt(encrypted_text, key)
print("Decrypted text:", decrypted_text)

Encrypted text: RSSRDE
Decrypted text: ATTACK
